In [1]:
import os
from tqdm.notebook import tqdm
import pandas
import numpy
import matplotlib.pyplot as plt

from ProblemInstance import ProblemInstance
from baseline_intra import bl_intra_1
from baseline_request import bl_req_FCFS
from baseline_pslice import bl_pslice
from baseline_pimlet import bl_pimlet
from our_approach import leader_follower

In [2]:
fontsize = 15
neurosim_dir = "NetStructure/"
summary_file = "NeuroSIMOut/summary.csv"
figure_dir = "figures/"
color_map = ["#ff8884", "indianred", "#f8ac8c",  "#2878b5", '#80b1d3', "#9ac9db"]
predef_N_X = numpy.array([12000, 400, 15600, 12000, 3600, 3600, 3600, 512, 90808, 
                          15608, 15608, 15200, 1000, 1000, 1000, 1000, 1000, 1000,])

In [3]:
def get_time(problem, plan): 
    return problem.running_time(*problem.get_pc_packages_from_plan(*plan))

In [ ]:
seed_set = numpy.arange(100)
n_req = 1000

multi_model_problem = ProblemInstance(neurosim_dir)
multi_model_problem.read_nn_profile(summary_file)
N_M = multi_model_problem.N_M
nmrange = numpy.arange(2, N_M+1)

intra_times = numpy.zeros_like(nmrange).astype(float)
inter_times = numpy.zeros_like(nmrange).astype(float)
pslice_times = numpy.zeros_like(nmrange).astype(float)
pimlet_times = numpy.zeros_like(nmrange).astype(float)
our_times = numpy.zeros_like(nmrange).astype(float)


nn_ids = numpy.arange(N_M)
for seed in tqdm(seed_set):
    # print(f"random seed={seed}", "-"*100)
    numpy.random.seed(seed)
    numpy.random.shuffle(nn_ids)
    for i, n_m in enumerate(nmrange):
        nn = nn_ids[:n_m]
        multi_model_problem.get_exact_requests(nn_ids=nn, n_x=predef_N_X[nn])

        plan_intra = bl_intra_1(multi_model_problem)
        intra_times[i] += get_time(multi_model_problem, plan_intra)

        package_inter = bl_req_FCFS(multi_model_problem) 
        inter_times[i] += multi_model_problem.running_time(*package_inter)

        plan_pslice = bl_pslice(multi_model_problem)
        pslice_times[i] += get_time(multi_model_problem, plan_pslice)

        package_pimlet = bl_pimlet(multi_model_problem) 
        pimlet_times[i] += multi_model_problem.running_time(*package_pimlet)

        plan_our = leader_follower(multi_model_problem)
        our_times[i] += get_time(multi_model_problem, plan_our)

intra_times /= len(seed_set)
inter_times /= len(seed_set)
pslice_times /= len(seed_set)
pimlet_times /= len(seed_set)
our_times /= len(seed_set)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
plt.figure(figsize=(10, 2))
names = numpy.arange(2, N_M+1)
n_col = 5
width = 6
width_per_col = width/n_col
base_xticks = numpy.arange(len(names)) * (width+1.5)
plt.bar(base_xticks-width/2, intra_times, width=width_per_col, label="Intra", color=color_map[0])
plt.bar(base_xticks-width/2 + width_per_col, inter_times, width=width_per_col, label="inter", color=color_map[1])
plt.bar(base_xticks-width/2 + width_per_col*2, pslice_times, width=width_per_col, label="PSLICE", color=color_map[2])
plt.bar(base_xticks-width/2 + width_per_col*3, pimlet_times, width=width_per_col, label="PIMLet", color=color_map[3])
plt.bar(base_xticks-width/2 + width_per_col*4, our_times, width=width_per_col, label="PIMShare", color=color_map[4])

plt.xticks(ticks=base_xticks-width_per_col/2, labels=names,rotation=0, fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.gca().yaxis.offsetText.set_fontsize(fontsize)
plt.xlabel("Number of DNN models", fontsize=fontsize)
plt.ylabel("Processing Time (ns)", fontsize=fontsize)
plt.legend(loc=(-0.05, 1.2), ncol=5, fontsize=fontsize)
plt.ylim((0,2e10))
plt.grid(axis='y')

plt.show()

In [ ]:
seed = 5
n_req = 20000
n_req_step = 10
# n_req_start = n_req//n_req_step
n_req_start = 2000

multi_model_problem = ProblemInstance(neurosim_dir)
multi_model_problem.read_nn_profile(summary_file)
N_M = multi_model_problem.N_M
nxrange = numpy.arange(n_req_start, n_req+1, n_req//n_req_step)

intra_times_x = numpy.zeros_like(nxrange).astype(float)
inter_times_x = numpy.zeros_like(nxrange).astype(float)
pslice_times_x = numpy.zeros_like(nxrange).astype(float)
pimlet_times_x = numpy.zeros_like(nxrange).astype(float)
our_times_x = numpy.zeros_like(nxrange).astype(float)

for i, n_x in tqdm(enumerate(nxrange)):
    # print(f"{n_x} requests", "-"*50)
    multi_model_problem.get_random_requests(n_x, seed=seed)
    
    plan_intra = bl_intra_1(multi_model_problem)
    intra_times_x[i] += get_time(multi_model_problem, plan_intra)

    package_inter = bl_req_FCFS(multi_model_problem) 
    inter_times_x[i] += multi_model_problem.running_time(*package_inter)

    plan_pslice = bl_pslice(multi_model_problem)
    pslice_times_x[i] += get_time(multi_model_problem, plan_pslice)

    package_pimlet = bl_pimlet(multi_model_problem) 
    pimlet_times_x[i] += multi_model_problem.running_time(*package_pimlet)

    plan_our = leader_follower(multi_model_problem)
    our_times_x[i] += get_time(multi_model_problem, plan_our)

In [ ]:
plt.figure(figsize=(10, 2))
names = nxrange
n_col = 5
width = 6
width_per_col = width/n_col
base_xticks = numpy.arange(len(names)) * (width+1.5)
plt.bar(base_xticks-width/2, intra_times_x, width=width_per_col, label="Intra", color=color_map[0])
plt.bar(base_xticks-width/2 + width_per_col, inter_times_x, width=width_per_col, label="inter", color=color_map[1])
plt.bar(base_xticks-width/2 + width_per_col*2, pslice_times_x, width=width_per_col, label="PSLICE", color=color_map[2])
plt.bar(base_xticks-width/2 + width_per_col*3, pimlet_times_x, width=width_per_col, label="PIMLet", color=color_map[3])
plt.bar(base_xticks-width/2 + width_per_col*4, our_times_x, width=width_per_col, label="PIMShare", color=color_map[4])

plt.xticks(ticks=base_xticks-width_per_col/2, labels=[n if i%2==0 else "" for i,n in enumerate(names)], rotation=0, fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.gca().yaxis.offsetText.set_fontsize(fontsize)
plt.xlabel("Number of requests", fontsize=fontsize)
plt.ylabel("Processing Time (ns)", fontsize=fontsize)
plt.legend(loc=(-0.05,1.2), ncol=5, fontsize=fontsize)
plt.ylim((0,15e9))
plt.grid(axis='y')

plt.show()